# Load Dependency


In [ ]:
!nvidia-smi

Sat Apr 22 17:31:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install scikit-tensor-py3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 55.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 27.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for numpy: filename=numpy-1.16.6-cp39-cp39-linux_x86_64.whl size=9574957 sha256=5be3d5bdfcbbfae5dbd46d9cd88eda5b5dbc7f8303e7b5472ca108c0c5a113c4
  Stored in directory: /root/.cache/pip/wheels/ce/72/38/377a1055ac9b575ebba22e507856aecdc37f9bf6372703e5f9
  Created wheel for scipy: filename=scipy-1.3.3-cp39-cp39-linux_x86_64.whl size=54851684 sha256=cb932111d6a771194a2c557bca01fb3003a4084c52797ad981c3e555798aab05
  Stored in directory: /root/.cache/pip/wheels/46/57/a5/

In [9]:
from google.colab import drive
import pandas as pd
import numpy as np
import random
import os
# from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
import joblib
#
# from keras.models import Sequential,Model
# from keras.layers import Dense,Input,Embedding,Concatenate,Reshape,Flatten
# from keras.layers import LSTM,Dropout,Activation,Bidirectional
# from keras.optimizers import Adagrad, Adam, SGD, RMSprop
# from keras.models import load_model,save_model
# from keras.utils import plot_model
#


import plotly.express as px

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM,Dropout,Activation,Input,Bidirectional
from tensorflow.keras.optimizers import Adagrad, Adam, SGD, RMSprop
from tensorflow.keras.models import load_model,save_model
from tensorflow.keras.utils import plot_model,to_categorical
# import tensorflow as tf
# from tensorflow import keras
# drive.mount('/content/gdrive/') 

In [8]:
from google.colab import drive
drive.mount('/content/drive')
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np


Mounted at /content/drive


# Load Data

In [110]:
%%time
# Load Raw Data
# https://drive.google.com/file/d/1-RXYMh61x9asaVU_w5tDHtq0CMmp_Rdn/view?usp=sharing
file_path_train = '/content/drive/MyDrive/MiniProject/TrainData.csv'
# https://drive.google.com/file/d/1--znLUhuYqoMMqQo1Gnf6WafrzzniIFL/view?usp=sharing
file_path_test ='/content/drive/MyDrive/MiniProject/TestData.csv'
cols = ['id','userId', 'gender', 'sport', 'calories', 'Route_id','derived_distance','altitude','derived_speed','heart_rate','distance','time_elapsed']
train_df = pd.read_csv(file_path_train,usecols=cols)
test_df = pd.read_csv(file_path_test,usecols=cols)
train_df.rename(columns={'derived_speed': 'speed'},inplace=True)

test_df.rename(columns={'derived_speed': 'speed'},inplace=True)
print(train_df.shape)
print(test_df.shape)

(50253, 12)
(12578, 12)
CPU times: user 24.1 s, sys: 7.91 s, total: 32 s
Wall time: 46.7 s


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [111]:
train_df.groupby(['Route_id','sport']).size()

Route_id  sport        
0         bike              4722
1         bike              2756
2         bike             10666
3         bike               788
4         bike               259
5         mountain bike      549
6         mountain bike      131
7         mountain bike     1941
8         run               3271
9         run               3353
10        run              17465
11        run                819
12        run               3089
13        run                444
dtype: int64

In [112]:
train_df.head(3)

,id,userId,gender,sport,calories,distance,altitude,heart_rate,derived_distance,speed,time_elapsed,Route_id
0,611012078,2568526,male,run,830.588,10.02,"[137.8, 137.8, 138.2, 138.8, 138.8, 138.6, 139...","[140, 140, 141, 149, 149, 150, 153, 157, 160, ...","[0.0016049429213742246, 0.01260242289257531, 0...","[5.777794516947209, 10.005378447215753, 10.428...","[1, 4, 5, 1, 5, 6, 11, 4, 6, 7, 8, 7, 7, 9, 5,...",9
1,326004692,11889307,unknown,bike,530.000,46.04,"[18.4, 17.6, 17.6, 17.6, 19.0, 19.8, 21.4, 22....","[80, 75, 68, 71, 78, 71, 67, 66, 64, 75, 68, 6...","[0.04356151376031798, 0.11363834298901357, 0.0...","[26.136908256190786, 34.09150289670407, 39.197...","[6, 12, 7, 11, 8, 10, 11, 8, 9, 13, 6, 13, 11,...",0
2,282989006,1912029,female,run,848.631,10.89,"[498.8, 498.4, 497.4, 495.2, 493.4, 491.0, 490...","[82, 93, 104, 117, 128, 135, 137, 139, 138, 14...","[0.010829722470982562, 0.010618715062935378, 0...","[7.797400179107444, 8.821489242357188, 8.31756...","[5, 4, 7, 8, 11, 7, 19, 6, 13, 13, 13, 11, 11,...",8


In [113]:
# Encode gender feature

conditions = [
    (train_df['gender'] == 'male'),
    (train_df['gender'] == 'female'),
    (train_df['gender'] == 'unknown')]

choices = [0, 1, 2]

train_df['genderId'] = np.select(conditions, choices, default=0)

conditions = [
    (test_df['gender'] == 'male'),
    (test_df['gender'] == 'female'),
    (test_df['gender'] == 'unknown')]

test_df['genderId'] = np.select(conditions, choices, default=0)

In [114]:
# Encode sport feature

conditions = [
    (train_df['sport'] == 'run'),
    (train_df['sport'] == 'bike'),
    (train_df['sport'] == 'mountain bike')]

choices = [0, 1, 2]

train_df['sportId'] = np.select(conditions, choices, default=0)

conditions = [
    (test_df['sport'] == 'run'),
    (test_df['sport'] == 'bike'),
    (test_df['sport'] == 'mountain bike')]

test_df['sportId'] = np.select(conditions, choices, default=0)

In [115]:
train_df.head(3)

,id,userId,gender,sport,calories,distance,altitude,heart_rate,derived_distance,speed,time_elapsed,Route_id,genderId,sportId
0,611012078,2568526,male,run,830.588,10.02,"[137.8, 137.8, 138.2, 138.8, 138.8, 138.6, 139...","[140, 140, 141, 149, 149, 150, 153, 157, 160, ...","[0.0016049429213742246, 0.01260242289257531, 0...","[5.777794516947209, 10.005378447215753, 10.428...","[1, 4, 5, 1, 5, 6, 11, 4, 6, 7, 8, 7, 7, 9, 5,...",9,0,0
1,326004692,11889307,unknown,bike,530.000,46.04,"[18.4, 17.6, 17.6, 17.6, 19.0, 19.8, 21.4, 22....","[80, 75, 68, 71, 78, 71, 67, 66, 64, 75, 68, 6...","[0.04356151376031798, 0.11363834298901357, 0.0...","[26.136908256190786, 34.09150289670407, 39.197...","[6, 12, 7, 11, 8, 10, 11, 8, 9, 13, 6, 13, 11,...",0,2,1
2,282989006,1912029,female,run,848.631,10.89,"[498.8, 498.4, 497.4, 495.2, 493.4, 491.0, 490...","[82, 93, 104, 117, 128, 135, 137, 139, 138, 14...","[0.010829722470982562, 0.010618715062935378, 0...","[7.797400179107444, 8.821489242357188, 8.31756...","[5, 4, 7, 8, 11, 7, 19, 6, 13, 13, 13, 11, 11,...",8,1,0


In [118]:
from sklearn.preprocessing import MinMaxScaler

# for features in sequence_df
for feature in train_sequence_df.columns:
    # Reshape data using array.reshape(-1, 1) if data has a single feature
    scaler = MinMaxScaler()
    train_sequence_df[feature] = train_sequence_df.apply(lambda x: np.concatenate(
        scaler.fit_transform(np.array(eval(x[feature])).reshape(-1, 1)), axis=0)[0:499], axis=1)
    
    test_sequence_df[feature] = test_sequence_df.apply(lambda x: np.concatenate(
        scaler.transform(np.array(eval(x[feature])).reshape(-1, 1)), axis=0)[0:499], axis=1)


# for features in context_df
for feature in train_context_df[train_context_df.columns.difference(['id','userId',
              'genderId', 
              'sportId', 
              'Route_id'])].columns:
    # Reshape data using array.reshape(-1, 1) if data has a single feature
    scaler = MinMaxScaler()
    train_context_df[feature] = scaler.fit_transform(
        train_context_df[feature].to_numpy().reshape(-1, 1))
    
    test_context_df[feature] = scaler.transform(
        test_context_df[feature].to_numpy().reshape(-1, 1))


ValueError: ignored

In [116]:
train_sequence_df = train_df[['altitude', 'heart_rate',
                       'derived_distance', 'speed']].copy()
train_context_df = train_df[['id','userId', 'genderId', 'sportId', 'calories', 'distance', 'Route_id']].copy()

test_sequence_df = test_df[['altitude', 'heart_rate',
                       'derived_distance', 'speed']].copy()
test_context_df = test_df[['id','userId', 'genderId', 'sportId', 'calories', 'distance', 'Route_id']].copy()

display(train_sequence_df.head(1))
display(train_context_df.head(1))

,altitude,heart_rate,derived_distance,speed
0,"[137.8, 137.8, 138.2, 138.8, 138.8, 138.6, 139...","[140, 140, 141, 149, 149, 150, 153, 157, 160, ...","[0.0016049429213742246, 0.01260242289257531, 0...","[5.777794516947209, 10.005378447215753, 10.428..."


,id,userId,genderId,sportId,calories,distance,Route_id
0,611012078,2568526,0,0,830.588,10.02,9


In [19]:
display(train_sequence_df.head(1))
display(train_context_df.head(1))

,altitude,heart_rate,derived_distance,speed
0,"[0.9747634069400635, 0.9747634069400635, 0.981...","[0.5714285714285715, 0.5714285714285715, 0.580...","[0.0, 0.2570558362253222, 0.3811466729812352, ...","[0.0, 0.42660865297738426, 0.4693170206773962,..."


,id,userId,genderId,sportId,calories,distance,Route_id
0,611012078,2568526,0,0,0.133587,0.068544,9


In [20]:
train_df = pd.concat([train_context_df, train_sequence_df], axis=1)
test_df = pd.concat([test_context_df, test_sequence_df], axis=1)

train_df.head(1)

,id,userId,genderId,sportId,calories,distance,Route_id,altitude,heart_rate,derived_distance,speed
0,611012078,2568526,0,0,0.133587,0.068544,9,"[0.9747634069400635, 0.9747634069400635, 0.981...","[0.5714285714285715, 0.5714285714285715, 0.580...","[0.0, 0.2570558362253222, 0.3811466729812352, ...","[0.0, 0.42660865297738426, 0.4693170206773962,..."


### Load embedding layer

In [21]:
# load embedding layer r = 13

routeEmbed_file_path = '/content/drive/MyDrive/MiniProject/Personalized_Intelligent_Fitness_Recommender-main/entity embedding/routeEmbed_tensorD_13.m'
userEmbed_file_path = '/content/drive/MyDrive/MiniProject/Personalized_Intelligent_Fitness_Recommender-main/entity embedding/userEmbed_tensorD_13.m'
# # load embedding layer r = 11
# routeEmbed_file_path = '/content/gdrive/My Drive/endomondoHR/Data/embedding/routeEmbed_tensorD_11.m'
# userEmbed_file_path = '/content/gdrive/My Drive/endomondoHR/Data/embedding/userEmbed_tensorD_11.m'
# # load embedding layer r = 2
# routeEmbed_file_path = '/content/gdrive/My Drive/endomondoHR/Data/embedding/routeEmbed_tensorD_2.m'
# userEmbed_file_path = '/content/gdrive/My Drive/endomondoHR/Data/embedding/userEmbed_tensorD_2.m'

route_embed_df=joblib.load(routeEmbed_file_path)
user_embed_df=joblib.load(userEmbed_file_path)
route_embed_dict = {row['Route_id']:list(row['routeEmbed']) for index,row in route_embed_df.iterrows()}
user_embed_dict = {row['userId']:list(row['userEmbed']) for index,row in user_embed_df.iterrows()}

In [22]:
def flatten_workout(df,workout_id):
  # The original sequencial data is in one cell of pd.DataFrame, and this function is to flatten it
  temp = df[df['id'] == workout_id].copy()
  temp.reset_index(drop=1,inplace=True)
  if len(temp) != 1:
    return None  
  context_info = ['id','userId','genderId','calories','sportId','Route_id']
  context_dict = {col:temp[col][0] for col in context_info}
  seq = []
  speed = list(temp['speed'])[0]
  altitude = list(temp['altitude'])[0]
  distance = list(temp['derived_distance'])[0]
  hr = list(temp['heart_rate'])[0]
  seq = [[altitude[i],distance[i],hr[i],speed[i]] for i in range(499)]
  sequence = np.array(seq)
  context = np.array([[context_dict['id'],context_dict['userId'],context_dict['genderId'],context_dict['sportId'],context_dict['Route_id'],context_dict['calories']]]*499)
  array = np.concatenate((context,
                sequence),axis=1)
  columns=['workoutId','userId','genderId','sportId','Route_id','calories','altitude','derived_distance','heart_rate','speed']
  df_sub = pd.DataFrame(array,columns=columns)
  df_sub = df_sub.astype({col:'float64' for col in columns if col not in ['gender','sport']})
  return df_sub

In [23]:
flatten_workout(train_df,train_df['id'][50249])

,workoutId,userId,genderId,sportId,Route_id,calories,altitude,derived_distance,heart_rate,speed
0,230506076.0,2568526.0,0.0,1.0,0.0,0.129091,0.048295,0.102514,0.000000,0.019794
1,230506076.0,2568526.0,0.0,1.0,0.0,0.129091,0.053977,0.083503,0.066667,0.037591
2,230506076.0,2568526.0,0.0,1.0,0.0,0.129091,0.053977,0.542649,0.093333,0.281373
3,230506076.0,2568526.0,0.0,1.0,0.0,0.129091,0.053977,0.266649,0.200000,0.341796
4,230506076.0,2568526.0,0.0,1.0,0.0,0.129091,0.053977,0.277506,0.266667,0.314266
...,...,...,...,...,...,...,...,...,...,...
494,230506076.0,2568526.0,0.0,1.0,0.0,0.129091,0.059659,0.180381,0.720000,0.747796
495,230506076.0,2568526.0,0.0,1.0,0.0,0.129091,0.034091,0.415899,0.773333,0.634597
496,230506076.0,2568526.0,0.0,1.0,0.0,0.129091,0.011364,0.548316,0.733333,0.608518
497,230506076.0,2568526.0,0.0,1.0,0.0,0.129091,0.000000,0.393595,0.733333,0.666787


### Generate Scaler
No need to run


In [ ]:
%%time
# 不需要跑
features = ['calories','distance','duration','time_elapsed','altitude','derived_distance','speed','heart_rate']
# features = ['calories','distance','duration','altitude','derived_distance','speed','heart_rate']
# get minmaxscaler for different features
features_dict = {f:[0] for f in features}
count = 0
for id in train_ids:
  if count % 5000==0:
    print(count)
  df_sub =flatten_workout(train_df,id)
  for feature in features:
    features_dict[feature].append(max(df_sub[feature]))
    features_dict[feature].append(min(df_sub[feature]))
  count+=1
features_mm = {k:MinMaxScaler().fit(np.array(features_dict[k]).reshape(-1,1)) for k,v in features_dict.items()}

NameError: ignored

In [ ]:
# save scaler model
# 不需要跑 
dir = '/content/gdrive/My Drive/endomondoHR/Data/scaler_model'
for k,model in features_mm.items():
  path = os.path.join(dir,'scaler_'+k+'_2.m')
  joblib.dump(model, path)

### Load Scaler

## Data Loader

In [24]:
# features we are going to use
features = ['calories','altitude','derived_distance','speed','heart_rate']
def convert_category(value,length):
  array_list = [0]*length
  array_list[int(value)] = 1
  return [array_list]*499

In [25]:
# generate scaled input
def generate_scaled(df_sub):
  gender = convert_category(df_sub['genderId'][0],3)
  sport = convert_category(df_sub['sportId'][0],3)
  route = [route_embed_dict[int(df_sub['Route_id'][0])]]*499
  user = [user_embed_dict[int(df_sub['userId'][0])]]*499
  feature_array = np.concatenate((gender,sport,route,user),axis=1)
  # feature_array = np.concatenate((gender,sport,user),axis=1)
  speed = []
  for k in features:
    # speed input
    if k == 'speed':
      speed = df_sub[k].values.reshape(-1,1)
    # heart rate input
    elif k == 'heart_rate': 
      hr = df_sub[k].values.reshape(-1,1)
    else:
      array = df_sub[k].values.reshape(-1,1)
      feature_array = np.concatenate((feature_array,array),axis=1)
  x1 = feature_array
  y1 = speed
  y2 = hr
  return x1.reshape(1,x1.shape[0],x1.shape[1]),y1.reshape(y1.shape[0],1),y2.reshape(y2.shape[0],1)

In [106]:
x,y1,y2 = generate_scaled(flatten_workout(train_df,	282989006))
print('Inputs shape:{}'.format(x.shape))

Inputs shape:(1, 499, 35)


In [27]:
# batch_generater
def batch_generator(df,batch_size=32):
  ids = list(df['id'])
  while True:
    X1,Y1,Y2= [],[],[]
    for id in random.sample(ids,batch_size):
      x1,y1,y2 = generate_scaled(flatten_workout(df,id))
      X1.append(x1[0])
      Y1.append(y1.flatten())
      Y2.append(y2.flatten())
      ids.remove(id)
    if len(ids) < batch_size:
      ids = list(df['id'])
    # yield np.array(X),[np.array(Y1),np.array(Y2),np.array(Y3)]
    yield np.array(X1),[np.array(Y1),np.array(Y2)]

# Modelling

## Build Model

In [ ]:
def single_lstm():
  input = Input(shape=(x.shape[1],x.shape[2]),name='Input')
  # lstm layer
  lstm_layer1 = LSTM(64,return_sequences=True,name='lstm1')(input)
  # dropout
  drop_hr = Dropout(0.2)(lstm_layer1)
  drop_speed = Dropout(0.2)(lstm_layer1)
  speed_output = Dense(1,activation='selu',name='speed_output')(drop_speed)
  hr_output = Dense(1,activation='selu',name='hr_output')(drop_hr)
  model = Model(inputs=input,outputs = [speed_output,hr_output])
  model.compile(loss={
                    'speed_output': 'mae',
                    'hr_output': 'mae'},
                loss_weights={
                    'speed_output': 1,
                    'hr_output':1
                },
                optimizer=RMSprop(lr=0.005))
  return model

In [ ]:
# single BiLSTM Model
def Bilstm():
  input = Input(shape=(x.shape[1],x.shape[2]),name='Input')
  # bilstm layer
  lstm_layer1 = Bidirectional(LSTM(64,return_sequences=True,name='lstm1'))(input)
  # dropout
  drop_hr = Dropout(0.2)(lstm_layer1)
  drop_speed = Dropout(0.2)(lstm_layer1)
  hr_output = Dense(1,activation='selu',name='hr_output')(drop_hr)
  speed_output = Dense(1,activation='selu',name='speed_output')(drop_speed)

  model = Model(inputs=input, outputs = [speed_output,hr_output])
  model.compile(loss={
                    'speed_output': 'mae',
                    'hr_output': 'mae'},
                loss_weights={
                    'speed_output': 1,
                    'hr_output':1
                },
                optimizer=RMSprop(lr=0.005))
  return model

In [ ]:
# 2 layer stacked LSTM model
def two_layer_LSTM():
  input = Input(shape=(x.shape[1],x.shape[2]),name='Input')
  # first lstm layer
  lstm_layer1 = LSTM(128,return_sequences=True,name='lstm1')(input)
  # dropout layer
  drop_hr = Dropout(0.2)(lstm_layer1)
  hr_output = Dense(1,activation='selu',name='hr_output')(drop_hr)
  # second lstm layer
  lstm_layer2 = LSTM(64,return_sequences=True,name='lstm2')(lstm_layer1)
  drop_speed = Dropout(0.2)(lstm_layer2)
  speed_output = Dense(1,activation='selu',name='speed_output')(drop_speed)

  model = Model(inputs=input, outputs = [speed_output,hr_output])
  model.compile(loss={
                    'speed_output': 'mae',
                    'hr_output': 'mae'},
                loss_weights={
                    'speed_output': 1,
                    'hr_output':1
                },
                optimizer=RMSprop(lr=0.005))
  return model

In [ ]:
#  2 layer stacked BiLSTM Model 
def two_layer_BiLSTM():
  input = Input(shape=(x.shape[1],x.shape[2]),name='Input')
  # first bilstm layer
  lstm_layer1 = Bidirectional(LSTM(64,return_sequences=True,name='lstm1'))(input)
  # dropout layer
  drop_hr = Dropout(0.2)(lstm_layer1)
  hr_output = Dense(1,activation='selu',name='hr_output')(drop_hr)
  # second bilstm layer
  lstm_layer2 = Bidirectional(LSTM(64,return_sequences=True,name='lstm2'))(lstm_layer1)
  drop_speed = Dropout(0.2)(lstm_layer2)
  speed_output = Dense(1,activation='selu',name='speed_output')(drop_speed)

  model = Model(inputs=input, outputs = [speed_output,hr_output])
  model.compile(loss={
                    'speed_output': 'mae',
                    'hr_output': 'mae'},
                loss_weights={
                    'speed_output': 1,
                    'hr_output':1
                },
                optimizer=RMSprop(lr=0.005))
  return model

In [ ]:
from tensorflow.keras.layers import Dense, Lambda, dot, Activation, concatenate
from tensorflow.keras.layers import Layer


class Attention(Layer):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def __call__(self, hidden_states):
        """
        Many-to-one attention mechanism for Keras.
        @param hidden_states: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        @author: felixhao28.
        """
        hidden_size = int(hidden_states.shape[2])
        score_first_part = Dense(hidden_size, use_bias=False)(hidden_states)
        h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,))(hidden_states)
        score = dot([score_first_part, h_t], [2, 1])
        attention_weights = Activation('softmax')(score)
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
        context_vector = dot([hidden_states, attention_weights], [1, 1])
        pre_activation = concatenate([context_vector, h_t])
        attention_vector = Dense(64, use_bias=True, activation='tanh')(pre_activation)
        return attention_vector


# USE heart rate predict speed
def two_layer_BiLSTM_Attention():
  input = Input(shape=(x.shape[1],x.shape[2]),name='Input')
  # first lstm layer
  lstm_layer1 = Bidirectional(LSTM(64,return_sequences=True,name='lstm1'))(input)
  # attention layer
  att_1 = Attention()(lstm_layer1)
  drop_hr = Dropout(0.2)(att_1)
  hr_output = Dense(1,activation='selu',name='hr_output')(att_1)

  lstm_layer2 = Bidirectional(LSTM(64,return_sequences=True,name='lstm2'))(lstm_layer1)
  att_2 = Attention()(lstm_layer2)
  drop_speed = Dropout(0.2)(att_2)
  speed_output = Dense(1,activation='selu',name='speed_output')(att_2)

  model = Model(inputs=input, outputs = [speed_output,hr_output])
  model.compile(loss={
                    'speed_output': 'mae',
                    'hr_output': 'mae'},
                loss_weights={
                    'speed_output': 1,
                    'hr_output':1
                },
                optimizer=Adagrad(lr=0.001))
  return model

## Train Model

In [ ]:
batch_size = 256
epochs = 50 
history = model.fit(batch_generator(train_df,batch_size),
            steps_per_epoch = len(train_df)/batch_size,
            validation_data = batch_generator(test_df,batch_size),
            validation_steps=1,epochs=epochs)

Epoch 1/50
196/196 [==============================] - 345s 2s/step - loss: 0.4023 - speed_output_loss: 0.2002 - hr_output_loss: 0.2021 - val_loss: 1.0965 - val_speed_output_loss: 0.2209 - val_hr_output_loss: 0.8755
Epoch 2/50
196/196 [==============================] - 330s 2s/step - loss: 0.3381 - speed_output_loss: 0.1794 - hr_output_loss: 0.1588 - val_loss: 1.0257 - val_speed_output_loss: 0.1982 - val_hr_output_loss: 0.8275
Epoch 3/50
196/196 [==============================] - 331s 2s/step - loss: 0.3305 - speed_output_loss: 0.1757 - hr_output_loss: 0.1548 - val_loss: 0.9318 - val_speed_output_loss: 0.1681 - val_hr_output_loss: 0.7637
Epoch 4/50
196/196 [==============================] - 333s 2s/step - loss: 0.3277 - speed_output_loss: 0.1744 - hr_output_loss: 0.1534 - val_loss: 0.8118 - val_speed_output_loss: 0.1612 - val_hr_output_loss: 0.6506
Epoch 5/50
196/196 [==============================] - 333s 2s/step - loss: 0.3260 - speed_output_loss: 0.1734 - hr_output_loss: 0.1525 - val

## Evaluate Model

In [80]:
model = load_model('/content/drive/MyDrive/MiniProject/Personalized_Intelligent_Fitness_Recommender-main/workout Speed&HeartRate model/Trained_Model/2Layer_BiLSTM_r=13.h5')

In [107]:
speed,hr=model.predict(x)

1/1 [==============================] - 0s 385ms/step


In [108]:
speed = speed.reshape(-1, 1)
speed=scaler.inverse_transform(speed)
speed

array([[30.611044],
       [30.054916],
       [29.873281],
       [32.403336],
       [29.88596 ],
       [36.63319 ],
       [31.624752],
       [38.266964],
       [33.770245],
       [34.594856],
       [33.639206],
       [33.98555 ],
       [33.54849 ],
       [31.857344],
       [34.129185],
       [32.53246 ],
       [33.099243],
       [31.853443],
       [32.509644],
       [32.644833],
       [32.424698],
       [31.995886],
       [32.545765],
       [30.286497],
       [31.254261],
       [29.531929],
       [30.055998],
       [29.20456 ],
       [26.266205],
       [22.746803],
       [23.584482],
       [25.28836 ],
       [27.262608],
       [26.96813 ],
       [25.617819],
       [27.84001 ],
       [26.704607],
       [26.677557],
       [26.335941],
       [26.766228],
       [25.499464],
       [26.742897],
       [25.904562],
       [26.208403],
       [24.929619],
       [26.322584],
       [24.893164],
       [25.9224  ],
       [25.158195],
       [25.225048],


In [109]:
hr=hr.reshape(-1,1)
hr=scaler.inverse_transform(hr)
hr

array([[64.248375],
       [71.12994 ],
       [78.79849 ],
       [74.52949 ],
       [82.02987 ],
       [76.28579 ],
       [82.548256],
       [75.88946 ],
       [78.11416 ],
       [78.23273 ],
       [78.18793 ],
       [77.056   ],
       [76.934326],
       [80.4436  ],
       [78.502014],
       [79.061554],
       [78.120255],
       [80.212074],
       [79.97413 ],
       [79.984436],
       [80.40763 ],
       [80.86871 ],
       [79.18489 ],
       [80.99839 ],
       [77.698395],
       [79.56896 ],
       [78.77584 ],
       [79.27596 ],
       [83.16428 ],
       [77.90285 ],
       [71.56185 ],
       [82.00477 ],
       [79.49322 ],
       [75.941765],
       [79.66964 ],
       [77.85488 ],
       [78.48628 ],
       [78.361145],
       [78.254364],
       [77.42656 ],
       [79.266556],
       [77.32827 ],
       [78.47334 ],
       [78.80603 ],
       [81.04264 ],
       [77.634735],
       [78.870575],
       [77.26878 ],
       [78.98903 ],
       [79.959465],


In [90]:
metrics=model.metrics_names
metrics

['loss', 'speed_output_loss', 'hr_output_loss']

In [88]:
batch_size = 256
model.evaluate_generator(generator=batch_generator(test_df,batch_size),steps=len(test_df)/batch_size,verbose=1)

<ipython-input-88-128d20604ad2>:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(generator=batch_generator(train_df,batch_size),steps=len(train_df)/batch_size,verbose=1)


196/196 [==============================] - 909s 5s/step - loss: 0.5497 - speed_output_loss: 0.3091 - hr_output_loss: 0.2406


[0.5497052669525146, 0.30911096930503845, 0.24059432744979858]

In [100]:
speed_loss = np.array([[0.309]])
speed_unscaled = scaler.inverse_transform(speed_loss)
print(speed_unscaled[0][0])

41.27691


In [101]:
hr_loss = np.array([[0.24059432744979858]])
hr_unscaled = scaler.inverse_transform(hr_loss)
print(hr_unscaled[0][0])

32.38485662519931


## Save Model

In [ ]:
#save model
dir = '/content/gdrive/My Drive/endomondoHR/Data/model/Final Model'
path = os.path.join(dir,'[1210]bilstm.h5')
model.save(path)
print(path)

/content/gdrive/My Drive/endomondoHR/Data/model/Final Model/[1115]single_user.h5


##Load Model


In [ ]:
# Load trained model：
model = load_model('/content/gdrive/My Drive/endomondoHR/Data/model/Final Model/[1210]bilstm.h5')
plot_model(model)